# 程式初始化

Import函數庫

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import backend as kr
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import optimizers
import logging
logging.getLogger('tensorflow').disabled = True

取得GPU資訊

In [ ]:
!/usr/local/cuda/bin/nvcc --version

!nvidia-smi

# 實作訓練資料產生函數

宣告Dobot位置與軸關節公式函數(順向運動學)

In [ ]:
def dobot_forword_kine(joints):

    if joints.ndim == 1:
        joints = np.expand_dims(joints, 0)

    q1 = joints[:, 0:1] # 0:1而非直接0->確保內部採用Column處理
    q2 = joints[:, 1:2]
    q3 = joints[:, 2:3]

    a2 = 135
    a3 = 147
    a4 = 61

    C1 = np.cos(q1)
    C2 = np.cos(q2)
    C23 = np.cos(q2 + q3)
    S1 = np.sin(q1)
    S2 = np.sin(q2)
    S23 = np.sin(q2 + q3)

    dx = C1 * (a3 * C23 + a2 * C2 + a4)
    dy = S1 * (a3 * C23 + a2 * C2 + a4)
    dz = -a2 * S2 - a3 * S23

    Point = np.hstack([dx, dy, dz]) # 建立陣列
    return Point

宣告訓練資料產生函數

In [ ]:
def gen_data(Train_num):
    joint_1 = (-np.pi / 2) + np.pi * np.random.rand(Train_num, 1)
    joint_2 = (-85 * np.pi / 180) + (85 * np.pi / 180) * np.random.rand(Train_num, 1)
    joint_3 = (-10 * np.pi / 180) + (105 * np.pi / 180) * np.random.rand(Train_num, 1)
    joints = np.hstack((joint_1, joint_2, joint_3))
    points = dobot_forword_kine(joints)
    return points, joints

產生訓練集資料

In [ ]:
x_train, y_train = gen_data(10000)

# 建立模型

模型架構參數

In [ ]:
num =   # 神經元數
lys =   # 隱藏層數
is_normalize = # 正規化

設定輸入層

In [ ]:
model = Sequential()
model.add(Dense(units=num, input_dim=3, activation='relu'))
if is_normalize:
    model.add(BatchNormalization())

設定隱藏層

In [ ]:
for i in range(0, lys):
    model.add(Dense(units=num, activation='relu'))
    if is_normalize:
        model.add(BatchNormalization())

設定輸出層

In [ ]:
model.add(Dense(units=3, activation='linear'))
if is_normalize:
    model.add(BatchNormalization())

# 訓練流程

自訂損失函數

In [ ]:
def euclidean_distance(y_true, y_pred):
        return kr.sqrt(kr.sum(kr.square(y_true - y_pred)))

學習參數

In [ ]:
lr =  # 學習率
loss_fnc = 'mse'  # 可選擇 'mae', 'mse', 'msle', 'categorical_crossentropy' 或自訂損失函數等...
# 可參考 https://www.tensorflow.org/api_docs/python/tf/keras/losses

編譯模型

In [ ]:
optm = optimizers.Adam(learning_rate=lr)

model.compile(optimizer = optm,
              loss=loss_fnc,  # 使用平均絕對值誤差作爲損失函數
              metrics=[euclidean_distance, 'accuracy'])  # 設定要儲存至model的損失
model.summary()

開始訓練

In [ ]:
batchsize = # 批次
epochs = # 訓練代數
validationsplit = # 驗證比例
history = model.fit(x_train, y_train,
          epochs=epochs,
          batch_size=batchsize,
          validation_split=validationsplit,
          verbose=1)

畫loss收斂圖

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'validation'], loc='center right')
plt.show()

畫正確率圖

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train', 'validation'], loc='center right')
plt.show()

# 自行測試

In [ ]:
x_test, y_test = gen_data(20)
y_pred = model.predict(x_test)

error_angles = euclidean_distance(y_test[0, :], y_pred[0, :])
error_points = euclidean_distance(x_test[0, :], dobot_forword_kine(y_pred)[0, :])

print('其中一個解:\n')
print(f'\t角度誤差: {error_angles},\n\t正確解: {y_test[0, :]}\n\t預測解: {y_pred[0, :]}\n')
print(f'\t座標誤差: {error_points},\n\t正確解: {x_test[0, :]}\n\t預測解: {dobot_forword_kine(y_pred)[0, :]}\n')

# 儲存與讀取模型

儲存模型

In [ ]:
# 儲存model爲'model.h5'
filePath = './model.h5'
model.save(filePath)  # 儲存model至'h5_filePath'

# 讀取model
loaded_model = load_model(filePath,
    custom_objects={'euclidean_distance': euclidean_distance})  # 若有設定自訂函數, 需將其加入custom_objects中

讀取模型

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(
    "./model.h5", custom_objects={'euclidean_distance': euclidean_distance})

def euclidean_distance(y_true, y_pred):
    return kr.sqrt(kr.sum(kr.square(y_true - y_pred)))

def dobot_ikine_from_DNN(model, x, y, z):
    ik_res = model.predict(np.array([[x, y, z]]), batch_size=500)
    return ik_res

# 使用DNN計算IK

In [ ]:
dobot_ik_test = dobot_ikine_from_DNN(model, 200, 0, 100)
print(np.degrees(dobot_ik_test))